In [ ]:
from sympy.physics.units import *
from sympy import *

# Rounding:
import decimal
from decimal import Decimal as DX
def iso_round(obj, pv, rounding=decimal.ROUND_HALF_EVEN):
    import sympy
    """
    Rounding acc. to DIN EN ISO 80000-1:2013-08
    place value = Rundestellenwert
    """
    assert pv in set([
        # place value   #  round to:
        100,            #  3rd last digit before decimal
        10,             #  2nd last
        1,              #  last
        0.1,            #  1st digit after decimal
        0.01,           #  2nd
        0.001,          #  3rd
        0.0001,         #  4th
        0.00001,        #  5th
        0.000001,       #  6th
        0.0000001,      #  7th
        0.00000001,     #  8th
        0.000000001,    #  9th
        0.0000000001,   # 10th
        ])
    try:
        tmp = DX(str(float(obj)))
        obj = tmp.quantize(DX(str(pv)), rounding=rounding)
    except:
        for i in range(len(obj)):
            tmp = DX(str(float(obj[i])))
            obj[i] = tmp.quantize(DX(str(pv)), rounding=rounding)
    return obj

# LateX:
kwargs = {}
kwargs["mat_str"] = "bmatrix"
kwargs["mat_delim"] = ""
# kwargs["symbol_names"] = {FB: "F^{\mathsf B}", }

# Units:
(k, M, G ) = ( 10**3, 10**6, 10**9 )
(mm, cm, deg) = ( m/1000, m/100, pi/180)
Newton = kg*m/s**2
Pa     = Newton/m**2
MPa    = M*Pa
GPa    = G*Pa
kN     = k*Newton

# ---

l, EI, q = var("l, EI, q")
a1, a2 = var("a1, a2")
x = var("x")

# Ansatz (must solve geometrical BCs):
xi = x/l

# case = Wellerdick's Solution:
case = 1

if case ==1:
    w = a1*(xi-xi*xi)
elif case == 2:
    t1 = a1*(xi - xi*xi)
    t2 = a2*(xi*xi - xi*xi*xi)
    w = t1 + t2
elif case == 3:
    t1 = a1*(xi - xi*xi)
    t2 = a2*(xi*xi*xi - xi*xi*xi*xi)
    w = t1 + t2
elif case == 4: # error is Wellerdick's key solution
    t1 = a1*(xi - xi*xi*xi)
    t2 = a2*(xi*xi*xi - xi*xi*xi*xi)
    w = t1 + t2

w1 = diff(w,  x)
w2 = diff(w1, x)

pprint("\nw(x):")
pprint(w)
pprint("\nw'(x):")
pprint(w1)
pprint("\nw''(x):")
pprint(w2)

pprint("\nUi:")
Ui = EI/2 * integrate(w2*w2, (x, 0, l))
pprint(Ui)

pprint("\nUa:")
Ua =  integrate(q*w, (x, 0, l))
pprint(Ua)

U = Ui - Ua

if case != 1:
    dUda1 = diff(U,a1)
    dUda2 = diff(U,a2)
    eq1 = Eq(dUda1,0)
    eq2 = Eq(dUda2,0)
    pprint("\nEq 1:")
    pprint(eq1)
    pprint("\nEq 2:")
    pprint(eq2)
    sol = solve([eq1,eq2], [a1,a2])
    a1s = sol[a1]
    a2s = sol[a2]
    pprint("\na1:")
    pprint(a1s)
    pprint("\na2:")
    pprint(a2s)
    w = w.subs({a1: a1s, a2:a2s})
else:
    dUda1 = diff(U,a1)
    eq1 = Eq(dUda1,0)
    pprint("\nEq 1:")
    pprint(eq1)
    sol = solve([eq1], [a1])
    a1s = sol[a1]
    pprint("\na1:")
    pprint(a1s)
    w = w.subs({a1: a1s})




w = w.simplify()
pprint("\nw(x):")
pprint(w)
pprint("\nw(l/2):")
tmp = w.subs(x,l/2)
pprint(tmp)

pprint("\nw(l/2) / w(l/2)_exact:")
exact = 5*q*l**4/(384*EI)
tmp /= exact
tmp = iso_round(tmp,0.01)
pprint(tmp)

# Case 1:
#
# w(x):
#    ⎛     2⎞
#    ⎜x   x ⎟
# a₁⋅⎜─ - ──⎟
#    ⎜l    2⎟
#    ⎝    l ⎠
#
# w'(x):
#    ⎛1   2⋅x⎞
# a₁⋅⎜─ - ───⎟
#    ⎜l     2⎟
#    ⎝     l ⎠
#
# w''(x):
# -2⋅a₁
# ──────
#    2
#   l
#
# Ui:
#        2
# 2⋅EI⋅a₁
# ────────
#     3
#    l
#
# Ua:
# a₁⋅l⋅q
# ──────
#   6
#
# Eq 1:
# 4⋅EI⋅a₁   l⋅q
# ─────── - ─── = 0
#     3      6
#    l
#
# a1:
#   4
#  l ⋅q
# ─────
# 24⋅EI
#
# w(x):
#  2
# l ⋅q⋅x⋅(l - x)
# ──────────────
#     24⋅EI
#
# w(l/2):
#   4
#  l ⋅q
# ─────
# 96⋅EI
#
# w(l/2) / w(l/2)_exact:
# 0.80
